# Implementation of panorama algorithm with RANSAC
20220259 전현빈

In [22]:
import cv2
import numpy as np
import os
import glob
import random
from typing import List, Tuple

def detect_features_and_match(img1: np.ndarray, img2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Detect SIFT features and find matches between two images."""
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = matcher.match(des1, des2)
    
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)
    
    return pts1, pts2

def custom_ransac(pts1: np.ndarray, pts2: np.ndarray, num_iterations: int = 1000, 
                  threshold: float = 10.0, min_inliers: int = 4) -> Tuple[np.ndarray, np.ndarray]:
    """Custom RANSAC to find best homography."""
    best_H = None
    best_inliers = []
    max_inliers = 0
    
    for _ in range(num_iterations):
        indices = random.sample(range(len(pts1)), 4)
        src_pts = pts1[indices]
        dst_pts = pts2[indices]
        
        H, _ = cv2.findHomography(src_pts, dst_pts, method=0)
        
        if H is None:
            continue
            
        inliers = []
        for i in range(len(pts1)):
            pt1 = np.array([pts1[i][0], pts1[i][1], 1])
            pt2 = pts2[i]
            
            projected = H @ pt1
            projected = projected[:2] / projected[2]
            
            dist = np.linalg.norm(projected - pt2)
            if dist < threshold:
                inliers.append(i)
        
        if len(inliers) > max_inliers and len(inliers) >= min_inliers:
            max_inliers = len(inliers)
            best_inliers = inliers
            best_H = H
    
    if not best_inliers:
        raise ValueError("RANSAC failed to find sufficient inliers")
        
    return best_H, np.array(best_inliers)

def compute_homography_with_inliers(pts1: np.ndarray, pts2: np.ndarray, 
                                  inliers: np.ndarray) -> np.ndarray:
    """Compute final homography using all inliers."""
    src_inliers = pts1[inliers]
    dst_inliers = pts2[inliers]
    H, _ = cv2.findHomography(src_inliers, dst_inliers, method=0)
    return H

def get_warped_image_size(images: List[np.ndarray], homographies: List[np.ndarray], 
                         ref_idx: int) -> Tuple[int, int, np.ndarray]:
    """Compute the size of the output panorama and the offset transform."""
    corners = []
    for i, img in enumerate(images):
        h, w = img.shape[:2]
        img_corners = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        
        # Apply homography relative to reference image
        if i == ref_idx:
            transformed_corners = img_corners
        else:
            H = np.eye(3)
            if i < ref_idx:
                # Chain homographies backward
                for j in range(i, ref_idx):
                    H = homographies[j] @ H
                transformed_corners = cv2.perspectiveTransform(img_corners, np.linalg.inv(H))
            else:
                # Chain homographies forward
                for j in range(ref_idx, i):
                    H = homographies[j] @ H
                transformed_corners = cv2.perspectiveTransform(img_corners, H)
        
        corners.append(transformed_corners)
    
    corners = np.concatenate(corners, axis=0)
    x_min, y_min = np.int32(corners.min(axis=0).ravel())
    x_max, y_max = np.int32(corners.max(axis=0).ravel())
    
    width = x_max - x_min
    height = y_max - y_min
    offset_transform = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]], dtype=np.float32)
    
    return width, height, offset_transform

def alpha_blending(img1: np.ndarray, img2: np.ndarray, mask1: np.ndarray, mask2: np.ndarray) -> np.ndarray:
    """Perform alpha blending for seamless stitching."""
    mask_sum = mask1 + mask2
    mask_sum[mask_sum == 0] = 1
    alpha1 = mask1 / mask_sum
    alpha2 = mask2 / mask_sum
    
    result = np.zeros_like(img1, dtype=np.float32)
    for c in range(3):
        result[:, :, c] = img1[:, :, c] * alpha1 + img2[:, :, c] * alpha2
    
    return result.astype(np.uint8)

def stitch_images(images: List[np.ndarray], ref_idx: int = 0) -> np.ndarray:
    """Stitch images into a panorama with the specified reference image."""
    if len(images) < 2:
        raise ValueError("At least two images are required")
    
    if ref_idx < 0 or ref_idx >= len(images):
        raise ValueError("Invalid reference index")
    
    # Convert images to grayscale
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    
    # Compute homographies between consecutive images
    homographies = []
    for i in range(len(images) - 1):
        pts1, pts2 = detect_features_and_match(gray_images[i+1], gray_images[i])
        H, inliers = custom_ransac(pts1, pts2)
        H = compute_homography_with_inliers(pts1, pts2, inliers)
        homographies.append(H)
    
    # Compute output size and offset
    width, height, offset_transform = get_warped_image_size(images, homographies, ref_idx)
    
    # Initialize panorama
    panorama = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Warp and blend images
    for i, img in enumerate(images):
        if i == ref_idx:
            # Warp reference image
            H_total = offset_transform
        else:
            H = np.eye(3)
            if i < ref_idx:
                # Chain homographies backward
                for j in range(i, ref_idx):
                    H = homographies[j] @ H
                H_total = offset_transform @ np.linalg.inv(H)
            else:
                # Chain homographies forward
                for j in range(ref_idx, i):
                    H = homographies[j] @ H
                H_total = offset_transform @ H
        
        # Warp image and mask
        warped = cv2.warpPerspective(img, H_total, (width, height))
        mask = cv2.warpPerspective(np.ones_like(gray_images[i], dtype=np.float32), 
                                 H_total, (width, height))
        
        # Blend
        if i == 0:
            panorama = warped
            panorama_mask = mask
        else:
            panorama = alpha_blending(panorama, warped, panorama_mask, mask)
            panorama_mask = np.clip(panorama_mask + mask, 0, 1)
    
    return panorama

# Main execution
if __name__ == "__main__":
    # 1. Specify image folder path
    image_dir = "/Users/jeonhyeonbin/Documents/CSED551/assn1/assn3/images"
    
    # 2. Load all images using glob
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.*")))
    
    # 3. Read images with OpenCV
    images = [cv2.imread(p) for p in image_paths]
    
    # Check if images are loaded correctly
    if not image_paths or any(img is None for img in images):
        raise ValueError("Failed to load images from the specified directory")
    
    # 4. Create panorama with first image as reference
    panorama = stitch_images(images, ref_idx=0)
    
    # 5. Save and display result
    cv2.imwrite("panorama_output.jpg", panorama)
    cv2.imshow("Panorama", panorama)

/var/folders/jz/79h17zh14cqbn2c_95yq690h0000gn/T/ipykernel_66797/4191210389.py:45: RuntimeWarning: divide by zero encountered in divide
  projected = projected[:2] / projected[2]
/var/folders/jz/79h17zh14cqbn2c_95yq690h0000gn/T/ipykernel_66797/4191210389.py:45: RuntimeWarning: invalid value encountered in divide
  projected = projected[:2] / projected[2]


# No Blending

In [25]:
import cv2
import numpy as np
import os
import glob
import random
from typing import List, Tuple

def detect_features_and_match(img1: np.ndarray, img2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Detect SIFT features and find matches between two images."""
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    matcher = cv2.BFMatcher(cv2.NORM_L2)
    matches = matcher.knnMatch(des1, des2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good_matches.append(m)
    
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)
    
    return pts1, pts2

def custom_ransac(pts1: np.ndarray, pts2: np.ndarray, num_iterations: int = 3000, 
                  threshold: float = 10.0, min_inliers: int = 4) -> Tuple[np.ndarray, np.ndarray]:
    """Custom RANSAC to find best homography."""
    best_H = None
    best_inliers = []
    max_inliers = 0
    
    for _ in range(num_iterations):
        indices = random.sample(range(len(pts1)), 4)
        src_pts = pts1[indices]
        dst_pts = pts2[indices]
        
        H, _ = cv2.findHomography(src_pts, dst_pts, method=0)
        
        if H is None:
            continue
            
        inliers = []
        for i in range(len(pts1)):
            pt1 = np.array([pts1[i][0], pts1[i][1], 1])
            pt2 = pts2[i]
            
            projected = H @ pt1
            projected = projected[:2] / projected[2]
            
            dist = np.linalg.norm(projected - pt2)
            if dist < threshold:
                inliers.append(i)
        
        if len(inliers) > max_inliers and len(inliers) >= min_inliers:
            max_inliers = len(inliers)
            best_inliers = inliers
            best_H = H
    
    if not best_inliers:
        raise ValueError("RANSAC failed to find sufficient inliers")
        
    return best_H, np.array(best_inliers)

def compute_homography_with_inliers(pts1: np.ndarray, pts2: np.ndarray, 
                                  inliers: np.ndarray) -> np.ndarray:
    """Compute final homography using all inliers."""
    src_inliers = pts1[inliers]
    dst_inliers = pts2[inliers]
    H, _ = cv2.findHomography(src_inliers, dst_inliers, method=0)
    return H

def get_warped_image_size(images: List[np.ndarray], homographies: List[np.ndarray], 
                         ref_idx: int) -> Tuple[int, int, np.ndarray]:
    """Compute the size of the output panorama and the offset transform."""
    corners = []
    for i, img in enumerate(images):
        h, w = img.shape[:2]
        img_corners = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        
        if i == ref_idx:
            transformed_corners = img_corners
        else:
            H = np.eye(3)
            if i < ref_idx:
                for j in range(i, ref_idx):
                    H = homographies[j] @ H
                transformed_corners = cv2.perspectiveTransform(img_corners, np.linalg.inv(H))
           
            else:
                for j in range(ref_idx, i):
                    H = homographies[j] @ H
                transformed_corners = cv2.perspectiveTransform(img_corners, H)
        
        corners.append(transformed_corners)
    
    corners = np.concatenate(corners, axis=0)
    x_min, y_min = np.int32(corners.min(axis=0).ravel())
    x_max, y_max = np.int32(corners.max(axis=0).ravel())
    
    width = x_max - x_min
    height = y_max - y_min
    offset_transform = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]], dtype=np.float32)
    
    return width, height, offset_transform

def color_correction(img: np.ndarray, ref_img: np.ndarray) -> np.ndarray:
    """Adjust image color in LAB space to match reference image."""
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    ref_lab = cv2.cvtColor(ref_img, cv2.COLOR_BGR2LAB)
    corrected_lab = img_lab.copy()
    for c in range(3):
        img_mean = np.mean(img_lab[:, :, c])
        ref_mean = np.mean(ref_lab[:, :, c])
        if img_mean != 0:
            corrected_lab[:, :, c] = np.clip(img_lab[:, :, c] * (ref_mean / img_mean), 0, 255)
    return cv2.cvtColor(corrected_lab, cv2.COLOR_LAB2BGR)

def simple_composite(img1: np.ndarray, img2: np.ndarray, mask1: np.ndarray, mask2: np.ndarray) -> np.ndarray:
    """Combine images without blending, using masks to overwrite."""
    # Create binary masks
    mask1_bin = (mask1 > 0).astype(np.uint8)
    mask2_bin = (mask2 > 0).astype(np.uint8)
    
    # Prioritize img2 in overlapping regions
    result = img1.copy()
    overlap_mask = mask2_bin.astype(bool)
    result[overlap_mask] = img2[overlap_mask]
    
    return result

def stitch_images(images: List[np.ndarray], ref_idx: int = 0) -> np.ndarray:
    """Stitch images into a panorama without blending."""
    if len(images) < 2:
        raise ValueError("At least two images are required")
    
    if len(images) == 2:
        gray_images = [cv2.equalizeHist(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in images]
        pts1, pts2 = detect_features_and_match(gray_images[1], gray_images[0])
        H, inliers = custom_ransac(pts1, pts2, num_iterations=3000, threshold=10.0, min_inliers=4)
        H = compute_homography_with_inliers(pts1, pts2, inliers)
        
        width, height, offset_transform = get_warped_image_size(images, [H], ref_idx)
        panorama = np.zeros((height, width, 3), dtype=np.uint8)
        
        for i, img in enumerate(images):
            if i != ref_idx:
                img = color_correction(img, images[ref_idx])
            
            H_total = offset_transform if i == ref_idx else offset_transform @ H
            warped = cv2.warpPerspective(img, H_total, (width, height))
            mask = cv2.warpPerspective(np.ones_like(gray_images[i], dtype=np.float32), 
                                     H_total, (width, height))
            # No Gaussian blur to emphasize seam
            if i == 0:
                panorama = warped
                panorama_mask = mask
            else:
                panorama = simple_composite(panorama, warped, panorama_mask, mask)
                panorama_mask = np.clip(panorama_mask + mask, 0, 1)
        return panorama
    
    first_pair = stitch_images(images[:2], ref_idx=0)
    remaining = [first_pair] + images[2:]
    return stitch_images(remaining, ref_idx=0)

# Main execution
if __name__ == "__main__":
    image_dir = "/Users/jeonhyeonbin/Documents/CSED551/assn1/assn3/images"
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.*")))
    images = [cv2.imread(p) for p in image_paths]
    
    if not image_paths or any(img is None for img in images):
        raise ValueError("Failed to load images from the specified directory")
    
    panorama = stitch_images(images, ref_idx=0)
    
    cv2.imwrite("panorama_no_blending.jpg", panorama)
    cv2.imshow("Panorama (No Blending)", panorama)


/var/folders/jz/79h17zh14cqbn2c_95yq690h0000gn/T/ipykernel_66797/3259214126.py:50: RuntimeWarning: invalid value encountered in divide
  projected = projected[:2] / projected[2]
/var/folders/jz/79h17zh14cqbn2c_95yq690h0000gn/T/ipykernel_66797/3259214126.py:50: RuntimeWarning: divide by zero encountered in divide
  projected = projected[:2] / projected[2]
